#### 层次化索引
层次化索引（hierarchical indexing）是pandas的一项重要功能，它使你能在一个轴上拥有多个（两个以上）索引级别

In [3]:
import pandas as pd
import numpy as np
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    1.740556
   2    0.170022
   3    0.336574
b  1   -0.663763
   3   -1.002070
c  1    0.914627
   2    1.951203
d  2    0.090787
   3   -0.467644
dtype: float64

In [4]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [6]:
data['b']

1   -0.663763
3   -1.002070
dtype: float64

In [7]:
data['b':'c']

b  1   -0.663763
   3   -1.002070
c  1    0.914627
   2    1.951203
dtype: float64

In [9]:
data.loc[['b','d']]

b  1   -0.663763
   3   -1.002070
d  2    0.090787
   3   -0.467644
dtype: float64

In [13]:
# 可以在“内层”中进行选取
data.loc[:,2]

a    0.170022
c    1.951203
d    0.090787
dtype: float64

层次化索引在数据重塑和基于分组的操作（如透视表生成）中扮演着重要的角色。例如，可以通过unstack方法将这段数据重新安排到一个DataFrame中

In [14]:
data.unstack()

,1,2,3
a,1.740556,0.170022,0.336574
b,-0.663763,NaN,-1.002070
c,0.914627,1.951203,NaN
d,NaN,0.090787,-0.467644


unstack的逆运算是stack

In [16]:
data.unstack().stack()

a  1    1.740556
   2    0.170022
   3    0.336574
b  1   -0.663763
   3   -1.002070
c  1    0.914627
   2    1.951203
d  2    0.090787
   3   -0.467644
dtype: float64

对于一个DataFrame，每条轴都可以有分层索引

In [17]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                      index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                      columns=[['Ohio', 'Ohio', 'Colorado'],
                               ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [22]:
# 可以给各层索引赋名字
frame.index.names = ['key1','key2']
frame.columns.names = ['state','color']

In [23]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [24]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

可以单独创建MultiIndex然后复用

In [42]:
cols = pd.MultiIndex.from_arrays([["水果", "水果", "蔬菜", "蔬菜"], ["苹果", "葡萄", "白菜", "萝卜"]],
                                names=['行业','品类'])
rows = pd.MultiIndex.from_arrays([["上半年", "上半年", "下半年", "下半年"], ["第一季度", "第二季度", "第三季度", "第四季度"]],
                               names=['半年度','季度'])
rs = pd.DataFrame(np.random.random(size=(4, 4)), index=rows,columns=cols)
rs

行业              水果                  蔬菜          
品类              苹果        葡萄        白菜        萝卜
半年度 季度                                          
上半年 第一季度  0.960451  0.792424  0.145780  0.762534
    第二季度  0.881338  0.787786  0.784893  0.321007
下半年 第三季度  0.031213  0.915891  0.674745  0.425477
    第四季度  0.886947  0.773714  0.796921  0.244964

#### 重排与分级排序

In [34]:
# swaplevel接受两个级别编号或名称，并返回一个互换了级别的新对象（但数据不会发生变化
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [40]:
# sort_index则根据单个级别中的值对数据进行排序
frame.sort_index(level=1)#级别从0开始

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [41]:
# 交换级别时，常常也会用到sort_index，这样最终结果就是按照指定顺序进行字母排序
frame.swaplevel(0,1).sort_index(1)

state     Colorado  Ohio    
color        Green Green Red
key2 key1                   
1    a           2     0   1
2    a           5     3   4
1    b           8     6   7
2    b          11     9  10

#### 根据级别汇总统计

In [43]:
rs

行业              水果                  蔬菜          
品类              苹果        葡萄        白菜        萝卜
半年度 季度                                          
上半年 第一季度  0.960451  0.792424  0.145780  0.762534
    第二季度  0.881338  0.787786  0.784893  0.321007
下半年 第三季度  0.031213  0.915891  0.674745  0.425477
    第四季度  0.886947  0.773714  0.796921  0.244964

In [45]:
# 半年度各品类零售额
rs.sum(level='半年度')

行业         水果                  蔬菜          
品类         苹果        葡萄        白菜        萝卜
半年度                                        
上半年  1.841789  1.580210  0.930673  1.083542
下半年  0.918160  1.689605  1.471666  0.670441

In [47]:
# 各行业销售
rs.sum(level='行业',axis=1)

行业              水果        蔬菜
半年度 季度                      
上半年 第一季度  1.752876  0.908315
    第二季度  1.669124  1.105900
下半年 第三季度  0.947103  1.100222
    第四季度  1.660662  1.041886

In [48]:
# 各行业各半年度销售情况
rs.sum(level='半年度').sum(level='行业',axis=1)

行业,水果,蔬菜
半年度,,
上半年,3.422000,2.014215
下半年,2.607765,2.142107


In [65]:
# pd.DataFrame(np.random.random(size=(4, 4)))
# pd.DataFrame(np.random.randn(4,4))
# pd.DataFrame(np.random.rand(4,4))
# pd.DataFrame(np.arange(4*4).reshape(4,4))

,0,1,2,3
0,0.950003,0.645686,0.825565,0.563120
1,0.840589,0.178065,0.267765,0.950875
2,0.717338,0.748865,0.354754,0.807178
3,0.240231,0.895007,0.729743,0.070554


#### 对DataFrame列进行索引

In [66]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                       'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                       'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [67]:
# DataFrame的set_index函数会将其一个或多个列转换为行索引，并创建一个新的DataFrame
frame2 = frame.set_index(['c','d'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [68]:
# 默认情况下，那些列会从DataFrame中移除，但也可以将其保留下来
frame.set_index(['c','d'],drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [71]:
# reset_index的功能跟set_index刚好相反，层次化索引的级别会被转移到列里面
frame2.reset_index().sort_index(axis=1)

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


#### 合并数据集
- merge可以根据一个或者多个键将不同的DataFrame连接起来，其实现方式与数据库的join操作类似
- concat可以沿着一条轴将多个DataFrame堆叠在一起
- combine_first可以将重复数据拼接在一起，用一个对象中的值填充另一个对象中的缺失值

In [82]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                     'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                     'data2': range(3)})
print(df1)
df2

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6


,key,data2
0,a,0
1,b,1
2,d,2


In [74]:
# 未指定对那一列进行连接时，merge会将重叠列的列名当做键
df1.merge(df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [76]:
df1.merge(df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [77]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [80]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                     'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                     'data2': range(3)})

如果两个对象的列名不同，可以分别指定键

In [81]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


- 默认情况下，merge为‘内连接’，结果为键的交集，其他方式还有"left"、"right"以及"outer"

In [83]:
pd.merge(df1,df2,how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


- 多对多产生的是笛卡尔积。如果左边的DataFrame有3个"b"行，右边的有2个，所以最终结果中就有6个"b"行

In [84]:
df5 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                     'data1': range(6)})

df6 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                     'data2': range(5)})
pd.merge(df5,df6,on='key',how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [85]:
pd.merge(df5,df6,on='key',how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


- 根据多个键进行合并，传入一个由列名组成的列表即可

In [88]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                      'key2': ['one', 'two', 'one'],
                      'lval': [1, 2, 3]})

right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                       'key2': ['one', 'one', 'one', 'two'],
                       'rval': [4, 5, 6, 7]})
pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


- 对于合并运算需要考虑的最后一个问题是对重复列名的处理,merge有一个实用的suffixes选项，用于指定附加到左右两个DataFrame对象的重叠列名上的字符串

In [93]:
pd.merge(left,right,on='key1').sort_index(axis=1)

,key1,key2_x,key2_y,lval,rval
0,foo,one,one,1,4
1,foo,one,one,1,5
2,foo,two,one,2,4
3,foo,two,one,2,5
4,bar,one,one,3,6
5,bar,one,two,3,7


In [92]:
pd.merge(left,right,on='key1',suffixes=('_left','_right')).sort_index(axis=1)

,key1,key2_left,key2_right,lval,rval
0,foo,one,one,1,4
1,foo,one,one,1,5
2,foo,two,one,2,4
3,foo,two,one,2,5
4,bar,one,one,3,6
5,bar,one,two,3,7


#### 索引上的合并
一些情况，DataFrame中的连接键位于其索引中。在这种情况下，可以传入left_index=True或right_index=True（或两个都传），说明索引被用作连接键

In [95]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                       'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
pd.merge(left1,right1,left_on='key',right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


对于层次化索引的数据，略微复杂，因为索引的合并默认是多键合并

In [96]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                                'Nevada', 'Nevada'],
                       'key2': [2000, 2001, 2002, 2001, 2002],
                       'data': np.arange(5.)})

righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                       index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                               'Ohio', 'Ohio'],
                              [2001, 2000, 2000, 2000, 2001, 2002]],
                       columns=['event1', 'event2'])
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [98]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [99]:
# 这种情况下，需要指明用作合并键的多个列
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [100]:
# 用how='outer'对重复索引值的处理
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True,how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [101]:
# 合并双方的索引
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                      index=['a', 'c', 'e'],
                      columns=['Ohio', 'Nevada'])

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                       index=['b', 'c', 'd', 'e'],
                       columns=['Missouri', 'Alabama'])
pd.merge(left2,right2,left_index=True,right_index=True,how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


- DataFrame还有一个便捷的join实例方法，它能更为方便地实现按索引合并。它还可用于合并多个带有相同或相似索引的DataFrame对象，但要求没有重叠的列
- 历史版本的遗留原因，DataFrame的join方法默认使用的是左连接

In [102]:
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [104]:
left1.join(right1,on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


#### 轴向连接
另一种数据合并运算也被称作连接（concatenation）、绑定（binding）或堆叠（stacking）。NumPy的concatenation函数可以用NumPy数组来实现

In [106]:
arr = np.arange(12).reshape(3,4)
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [107]:
np.concatenate([arr,arr])

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [108]:
arr[1]

array([4, 5, 6, 7])

In [110]:
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [111]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [112]:
pd.concat([s1,s2,s3],axis=1)

D:\software\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


这种情况下，另外的轴上没有重叠，从索引的有序并集（外连接）上就可以看出来。传入join='inner'即可得到它们的交集

In [116]:
s4 = pd.concat([s1,s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [126]:
pd.concat([s1,s4],axis=1,join='inner')

,0,1
a,0,0
b,1,1


In [129]:
# 可以通过join_axes指定要在其它轴上使用的索引
pd.concat([s1,s4],join_axes=[['a''b','e']])
# pd.concat([s1,s4],join_axes=['a','b','e'])

AssertionError: length of join_axes must be equal to 0

- 参与连接的片段在结果中区分不开。假设你想要在连接轴上创建一个层次化索引。使用keys参数即可达到这个目的

In [135]:
result = pd.concat([s1,s2,s3],keys=['one','two','three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

- 如果沿着axis=1对Series进行合并，则keys就会成为DataFrame的列名

In [134]:
pd.concat([s1,s2,s3],axis=1,keys=['one','two','three'])

D:\software\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [136]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
pd.concat([df1, df2])

D:\software\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,a,b,c,d
0,0.126421,-0.585494,-1.904056,-0.809372
1,-0.052163,1.470523,-1.329825,0.767693
2,0.084963,-0.206690,0.680219,1.307993
0,-0.188437,-2.006707,NaN,0.024497
1,-0.707296,-0.351263,NaN,0.123625


- 合并后DataFrame的行索引存在重复，不利于后期数据操作,传入ignore_index=True即可对行索引进行重置

In [137]:
pd.concat([df1, df2], ignore_index=True)

D:\software\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,0.126421,-0.585494,-1.904056,-0.809372
1,-0.052163,1.470523,-1.329825,0.767693
2,0.084963,-0.206690,0.680219,1.307993
3,-0.188437,-2.006707,NaN,0.024497
4,-0.707296,-0.351263,NaN,0.123625


#### 合并重叠数据

In [2]:
import pandas as pd
import numpy as np
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
               index=['f', 'e', 'd', 'c', 'b', 'a'])

b = pd.Series(np.arange(len(a), dtype=np.float64),
               index=['f', 'e', 'd', 'c', 'b', 'a'])
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [6]:
b[-1] = np.nan
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [7]:
# where函数，它表示一种等价于面向数组的if-else
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [8]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                     'b': [np.nan, 2., np.nan, 6.],
                     'c': range(2, 18, 4)})

df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                     'b': [np.nan, 3., 4., 6., 8.]})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [9]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [10]:
# combine_first传递对象中的数据为调用对象的缺失数据“打补丁”
df1.combine_first(df2)#用df2中的数据填补df1中相同位置为空的数据

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


#### 重塑层次化索引
层次化索引为DataFrame数据的重排任务提供了一种具有良好一致性的方式。主要功能有二：
- stack：将数据的列“旋转”为行索引。
- unstack：将数据的行索引“旋转”为列。

In [14]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                     index=pd.Index(['Ohio','Colorado'], name='state'),
                     columns=pd.Index(['one', 'two', 'three'],
                     name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [23]:

result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [22]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [24]:
# 默认情况下，unstack操作的是最内层（stack也是如此）,传入分层级别的编号或名称即可对其它级别进行unstack操作
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [26]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1,s2],keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [32]:
# 如果不是所有的级别值都能在各分组中找到的话，则unstack操作可能会引入缺失数据
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [34]:
# stack默认会滤除缺失数据，因此该运算是可逆的
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [37]:
# 也可以保留缺失数据索引
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [38]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [39]:
# 在对DataFrame进行unstack操作时，作为旋转轴的级别将会成为结果中的最低级别
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [40]:
# 调用stack，我们可以指明轴的名字
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

In [46]:
df.unstack('number').stack('side')

number          one  three  two
state    side                  
Ohio     left     0      2    1
         right    5      7    6
Colorado left     3      5    4
         right    8     10    9

In [43]:
df.unstack('number').stack('side').T

state  Ohio       Colorado      
side   left right     left right
number                          
one       0     5        3     8
three     2     7        5    10
two       1     6        4     9

In [95]:
data = pd.read_csv('E:/ML/data/macrodata.csv',sep='\s+')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [97]:
periods = pd.PeriodIndex(year=data.year,quarter=data.quarter,name='date')
columns = pd.Index(['realgdp','infl','unemp'],name='item')
data1 = data.reindex(columns=columns)
data1.index = periods.to_timestamp()
ldata = data1.stack().reset_index().rename(columns={0: 'value'})

In [98]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


#### 将宽格式旋转为长格式
旋转DataFrame的逆运算是pandas.melt,是合并多个列成为一个，产生一个比输入长的DataFrame

In [3]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                    'A': [1, 2, 3],
                    'B': [4, 5, 6],
                    'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [4]:
# 使用pandas.melt，我们必须指明哪些列是分组指标
melted = pd.melt(df,['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [5]:
# 使用pivot，可以重塑回原来的样子
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [7]:
# pivot(index,columns,value)的结果从列创建了一个索引，用作行标签，
# 可以使用reset_index将数据移回列,在获得新的index，原来的index变成数据列，保留下来。不想保留原来的index，使用参数 drop=True，默认 False
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [10]:
# 也可以指定自己，作为值的列
r = pd.melt(df,id_vars=['key'],value_vars=['A','B'])
r

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [15]:
# pandas.melt也可以不用分组指标
pd.melt(df,value_vars=['A','B','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [17]:
pd.melt(df,value_vars=['key','A','B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
